In [1]:
using Revise
using BilevelTrajOpt

In [4]:
# test the solution
function f(x)
    return 0.
end

function h(x)
    return x[1:2] .* x[3:4]
end

function g(x)
    return [2. - x[1], 3. - x[3]]
end

x0 = [1., 1., 1., 1.]
λ0 = ones(length(h(x0)))
μ0 = ones(length(g(x0)))

x_min = 0.*ones(length(x0))
x_max = 10.*ones(length(x0))

x_sol = auglag_solve(x0,λ0,μ0,f,h,g,x_min,x_max)

num_h = length(λ0)
num_g = length(μ0)
x_sol_ip = ip_solve(x0,f,h,g,num_h,num_g,x_min,x_max)

display(x_sol)
display(x_sol_ip)

([0.506102, 0.962335, 6.22887, 0.962381], [-1.06577e12, 4.99927e-13], [-1.06355e13, -2.07001e10])

4-element Array{Float64,1}:
 2.49972e-13
 1.31745e-6 
 3.00032    
 1.31738e-6 

Infeasible_Problem_Detected


In [117]:
# test the gradient
function solve_prob(z)
    function f(x)
#         return x[1] * x[4] * (x[1] + x[2] + x[3]) + x[3]
        return dot(x,x)
    end

    function h(x)
#         return [z[1] - (x[1]^2 + x[2]^2 + x[3]^2 + x[4]^2)]
        return z[1] - x[1] + x[4]
    end

    function g(x)
#         return [z[2] - (x[1]*x[2]*x[3]*x[4])]
        return z[2] - x[2] - x[3]*5
    end

    x0 = [1., 1., 1., 1.]
    λ0 = ones(length(h(x0)))
    μ0 = ones(length(g(x0)))

    x_min = 1.*ones(length(x0))
    x_max = 10.*ones(length(x0))

    x_sol = auglag_solve(x0,λ0,μ0,f,h,g,x_min,x_max)
    
    x_sol
end

solve_prob (generic function with 2 methods)

In [118]:
z0 = [2.,-1.]

# autodiff 
J_auto = ForwardDiff.jacobian(solve_prob,z0)

# numerical
ϵ = 1e-12
J_num = zeros(size(J_auto))
for i = 1:length(z0)
    δ = zeros(length(z0))
    δ[i] = ϵ
    J_num[:,i] = (solve_prob(z0 + δ) - solve_prob(z0))/ϵ
end

display(solve_prob(z0))

display(J_auto)
display(J_num)

4-element Array{Float64,1}:
 1.69309
 1.30305
 1.26506
 1.12693

4×2 Array{Float64,2}:
  0.249954   0.0      
  0.0        0.0100549
  0.0        0.0447752
 -0.0595996  0.0      

4×2 Array{Float64,2}:
  0.2498    0.0      
  0.0       0.0102141
  0.0       0.0450751
 -0.059508  0.0      